# Predicting the non-violent crime rate (total number of non-violent crimes per 100K population)

In [1]:
#installing dependencies 

!pip install Seaborn
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


# Data Loading and Preprocessing 

Loading data and cleaning it

In [2]:
#Google drive mount and importing data 
from google.colab import drive
drive.mount('/content/drive')

# load dataset
df = pd.read_csv('/content/drive/MyDrive/1.ASP2022/crime_ml.csv')

# drop unnecessary columns with string values 

df.drop(columns = ['communityname', 'state'], inplace=True, errors='raise')

#resetting indexes 
df = df.reset_index()


#finding out the number of null values in the dataset: I can't use mean or median here to replace them because of their nature  
print(df.isnull().sum())

#fill NaN values with 0 to let the machine know that these are missing values anyway 
df.fillna(0, inplace=True)

#checking again if missin values are removed 
print(df.isnull().sum())

# summarize shape
print(df.shape)
# summarize first few lines
df.head()
#checks
df.describe()


  


Mounted at /content/drive
index                     0
countycode             1221
communitycode          1224
fold                      0
population                0
                       ... 
autotheftperpop           3
arsons                   91
arsonsperpop             91
violentcrimesperpop     221
nonviolperpop            97
Length: 146, dtype: int64
index                  0
countycode             0
communitycode          0
fold                   0
population             0
                      ..
autotheftperpop        0
arsons                 0
arsonsperpop           0
violentcrimesperpop    0
nonviolperpop          0
Length: 146, dtype: int64
(2215, 146)


,index,countycode,communitycode,fold,population,householdsize,racepctblack,racepctwhite,racepctasian,racepcthisp,...,burglaries,burglperpop,larcenies,larcperpop,autotheft,autotheftperpop,arsons,arsonsperpop,violentcrimesperpop,nonviolperpop
count,2215.000000,2215.000000,2215.000000,2215.000000,2.215000e+03,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,...,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000
mean,1107.000000,29.432957,20226.802709,5.494357,5.311798e+04,2.707327,9.335102,83.979819,2.670203,7.950176,...,760.205869,1032.030523,2134.734086,3368.410780,515.992777,473.323689,29.637923,30.832695,530.304005,4693.298483
std,639.559744,85.392208,28188.955806,2.872924,2.046203e+05,0.334120,14.247156,16.419080,4.473843,14.589832,...,3109.719923,763.783732,7595.829308,1904.074527,3256.011544,504.625282,176.491345,38.952603,609.438219,2861.188528
min,0.000000,0.000000,0.000000,1.000000,1.000500e+04,1.600000,0.000000,2.680000,0.030000,0.120000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,553.500000,0.000000,0.000000,3.000000,1.436600e+04,2.500000,0.860000,76.319999,0.620000,0.930000,...,94.500000,510.069990,391.500000,2035.600000,30.000000,156.550000,1.000000,6.460000,113.130000,2725.990000
50%,1107.000000,0.000000,0.000000,5.000000,2.279200e+04,2.660000,2.870000,90.349998,1.230000,2.180000,...,205.000000,821.659970,745.000000,3077.530000,75.000000,301.549990,5.000000,19.750000,316.549990,4277.310100
75%,1660.500000,23.000000,41705.000000,8.000000,4.302400e+04,2.850000,11.145000,96.225002,2.670000,7.810000,...,507.500000,1348.670000,1673.000000,4334.675050,231.500000,589.225005,15.000000,41.435000,722.000000,6100.869900
max,2214.000000,840.000000,94597.000000,10.000000,7.322564e+06,5.280000,96.669998,99.629997,57.459999,95.290001,...,99207.000000,11881.020000,235132.000000,25910.551000,112464.000000,4968.589800,5119.000000,436.370000,4877.060100,27119.760000


Creating Arrays for the Features and the Response Variable

In [3]:
#creating an object of the target variable 
target_column= ['nonviolperpop']

#take all other features and exclude the target variable
predictors = list(set(list(df.columns))-set(target_column))

print(predictors)

#normalising the predictors because they might have significant different values
#here, normalization is done via scaling of the predictors between 0 and 1. 
df[predictors] = df[predictors]/df[predictors].max()

#finding out again if the number of null values in the dataset: I can't use mean or median here to replace them because of their nature  
print(df.isnull().sum())

#Remove missing values 
#df.fillna(0, inplace=True)

#checking again 
#print(df.isnull().sum())

#displaying the summary of the normalized data
df.describe()


['lemastotreqperpop', 'persperfam', 'pctkidsbornnevermar', 'pctyoungkids2par', 'pctpolicwhite', 'arsonsperpop', 'arsons', 'robberies', 'medowncostpctinc', 'communitycode', 'lemasgangunitdeploy', 'pcthousownocc', 'agepct12t21', 'violentcrimesperpop', 'pctlarghouseoccup', 'pctwsocsec', 'otherpercap', 'malepctnevmarr', 'racialmatchcommpol', 'landarea', 'numburban', 'rentmedian', 'assaultperpop', 'blackpercap', 'malepctdivorce', 'burglperpop', 'pctsamecity85', 'pctworkmom', 'assaults', 'pctfam2par', 'pctpersownoccup', 'pctemplmanu', 'policoperbudg', 'pctsamehouse85', 'percapinc', 'pcthousoccup', 'lemasswftfieldperpop', 'racepcthisp', 'burglaries', 'lemaspctpoliconpatr', 'pcthousnophone', 'lemasswornft', 'lemasswftperpop', 'policreqperoffic', 'medfaminc', 'pctspeakenglonly', 'persperoccuphous', 'ownoccmedval', 'autotheft', 'numinshelters', 'agepct65up', 'femalepctdiv', 'hisppercap', 'pctworkmomyoungkids', 'pctrecentimmig', 'pctvacantboarded', 'pctpolicblack', 'pctpolicminor', 'pctlarghousef

,index,countycode,communitycode,fold,population,householdsize,racepctblack,racepctwhite,racepctasian,racepcthisp,...,burglaries,burglperpop,larcenies,larcperpop,autotheft,autotheftperpop,arsons,arsonsperpop,violentcrimesperpop,nonviolperpop
count,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,...,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000
mean,0.500000,0.035039,0.213821,0.549436,0.007254,0.512751,0.096567,0.842917,0.046471,0.083431,...,0.007663,0.086864,0.009079,0.130002,0.004588,0.095263,0.005790,0.070657,0.108734,4693.298483
std,0.288871,0.101657,0.297990,0.287292,0.027944,0.063280,0.147379,0.164801,0.077860,0.153110,...,0.031346,0.064286,0.032305,0.073486,0.028952,0.101563,0.034478,0.089265,0.124960,2861.188528
min,0.000000,0.000000,0.000000,0.100000,0.001366,0.303030,0.000000,0.026900,0.000522,0.001259,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.000000,0.000000,0.300000,0.001962,0.473485,0.008896,0.766034,0.010790,0.009760,...,0.000953,0.042931,0.001665,0.078563,0.000267,0.031508,0.000195,0.014804,0.023196,2725.990000
50%,0.500000,0.000000,0.000000,0.500000,0.003113,0.503788,0.029689,0.906855,0.021406,0.022878,...,0.002066,0.069157,0.003168,0.118775,0.000667,0.060691,0.000977,0.045260,0.064906,4277.310100
75%,0.750000,0.027381,0.440870,0.800000,0.005876,0.539773,0.115289,0.965824,0.046467,0.081960,...,0.005116,0.113515,0.007115,0.167294,0.002058,0.118590,0.002930,0.094954,0.148040,6100.869900
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,27119.760000


K Fold analysis: splitting data into a training set and a testing set 

In [4]:
#splitting data 

X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

print(X_train.shape); print(X_test.shape)


(1550, 145)
(665, 145)


# The models

# LASSO ( Least Absolute Shrinkage and Selection Operator)

In [5]:
### ON THE TRAINING SET##

#Setting the alpha value 
model_lasso = Lasso(alpha=0.01)

#fitting the model to the training data
model_lasso.fit(X_train, y_train) 

#the actual prediction process 
pred_train_lasso= model_lasso.predict(X_train)

#evaluation metrics: R-sqaured and RMSE of course!
print('RMSE for training:', np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
print('R-Squared for training', r2_score(y_train, pred_train_lasso))

### ON THE TESTING SET##

#the actual prediction process 
pred_test_lasso= model_lasso.predict(X_test)

#evaluation metrics: R-sqaured and RMSE of course!
print('RMSE for testing:',np.sqrt(mean_squared_error(y_test,pred_test_lasso))) 
print('R-Squared for testing', r2_score(y_test, pred_test_lasso))


RMSE for training: 946.5026853670738
R-Squared for training 0.8856709777649749
RMSE for testing: 1273.6227599179683
R-Squared for testing 0.8194703995089188


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.817e+08, tolerance: 1.215e+06
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


# Ridge Regression 

In [6]:
### ON THE TRAINING SET##

#Setting the alpha value 
rr = Ridge(alpha=0.01)

#fitting the model to the training data
rr.fit(X_train, y_train) 

#the actual prediction process 
pred_train_rr= rr.predict(X_train)


#evaluation metrics: R-sqaured and RMSE of course!
print( 'RMSE for training:', np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print( 'R-Squared for training', r2_score(y_train, pred_train_rr))


### ON THE TESTING SET##

#the actual prediction process 
pred_test_rr= rr.predict(X_test)


#evaluation metrics: R-sqaured and RMSE of course!
print('RMSE for testing:',np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print('R-Squared for testing', r2_score(y_test, pred_test_rr))

RMSE for training: 953.4764907319002
R-Squared for training 0.8839800250950716
RMSE for testing: 1187.4263073582624
R-Squared for testing 0.8430793381520818


# ElasticNet Regression 

In [7]:
#Check above comments in other methods, it's pretty the same

#For the training set 

model_enet = ElasticNet(alpha = 0.01)
model_enet.fit(X_train, y_train) 
pred_train_enet= model_enet.predict(X_train)
print('RMSE for training:', np.sqrt(mean_squared_error(y_train,pred_train_enet)))
print('R-Squared for training:', r2_score(y_train, pred_train_enet))

# For the testing set 

pred_test_enet= model_enet.predict(X_test)
print('RMSE for testing:', np.sqrt(mean_squared_error(y_test,pred_test_enet)))
print('R-Squared for testing:', r2_score(y_test, pred_test_enet))



RMSE for training: 1387.5969024573192
R-Squared for training: 0.7542806081382679
RMSE for testing: 1585.8672342961765
R-Squared for testing: 0.7201015775630772


# Using K-fold regularisation parameters for each model

LASSO

In [8]:
from sklearn.linear_model import LassoCV

# Lasso with 5 fold cross-validation
model = LassoCV(cv=5, random_state=0, max_iter=10000)

# Fit model
model.fit(X_train, y_train)

LassoCV(cv=5, max_iter=10000, random_state=0)

# Here, this will show best value of penalization chosen by cross validation:

model.alpha_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2.1805591756714717

In [9]:
#Now predicting with the best alpha 

# Set best alpha
lasso_best = Lasso(alpha=model.alpha_)
lasso_best.fit(X_train, y_train)

#Show model coefficients and names:

#print(list(zip(lasso_best.coef_, X)))

#the actual prediction 
print('R squared training set', round(lasso_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_best.score(X_test, y_test)*100, 2))

R squared training set 86.61
R squared test set 86.15


Ridge Regression

In [10]:
from sklearn.linear_model import RidgeCV

# Ridge with 5 fold cross-validation
model = RidgeCV(cv=5)

# Fit model
model.fit(X_train, y_train)

RidgeCV(cv=5)

# Here, this will show best value of penalization chosen by cross validation:

model.alpha_

0.1

In [11]:
#Now predicting with the best alpha 

# Set best alpha
ridge_best = Ridge(alpha=model.alpha_)
ridge_best.fit(X_train, y_train)

#Show model coefficients and names:

#print(list(zip(lasso_best.coef_, X)))

#the actual prediction 
print('R squared training set', round(ridge_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(ridge_best.score(X_test, y_test)*100, 2))

R squared training set 87.9
R squared test set 85.61


Elastic Net Regression

In [12]:
from sklearn.linear_model import ElasticNetCV

# Lasso with 5 fold cross-validation
model = ElasticNetCV(cv=5, random_state=0, max_iter=10000)

# Fit model
model.fit(X_train, y_train)

ElasticNetCV(cv=5, max_iter=10000, random_state=0)

# Here, this will show best value of penalization chosen by cross validation:

model.alpha_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5014226332213989

In [13]:
#Now predicting with the best alpha 

# Set best alpha
ElasticNet_best = ElasticNet(alpha=model.alpha_)
ElasticNet_best.fit(X_train, y_train)

#Show model coefficients and names:

#print(list(zip(lasso_best.coef_, X)))

#the actual prediction 
print('R squared training set', round(ElasticNet_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(ElasticNet_best.score(X_test, y_test)*100, 2))

R squared training set 38.31
R squared test set 33.92
